In [3]:
import sqlite3
import requests

# Función para obtener el precio del bitcoin en USD, GBP y EUR desde el API de CoinDesk
def obtener_precio_bitcoin():
    try:
        response = requests.get("https://api.coindesk.com/v1/bpi/currentprice.json")
        data = response.json()
        return data['bpi']
    except Exception as e:
        print("Error al obtener el precio del bitcoin:", e)
        return None

# Función para obtener el tipo de cambio del PEN utilizando el servicio de la SUNAT
def obtener_tipo_cambio_pen():
    try:
        response = requests.get("https://api.sunat.cloud/cambio")
        data = response.json()
        tipo_cambio_pen = data["cambio"]["PEN"]
        return tipo_cambio_pen
    except Exception as e:
        print("Error al obtener el tipo de cambio de PEN:", e)
        return None

# Función para crear la tabla 'bitcoin' y almacenar los datos en la base de datos 'base.db'
def crear_tabla_y_almacenar_datos():
    # Obtener precios de bitcoin y tipo de cambio de PEN
    precios_bitcoin = obtener_precio_bitcoin()
    tipo_cambio_pen = obtener_tipo_cambio_pen()

    if precios_bitcoin and tipo_cambio_pen:
        try:
            conexion = sqlite3.connect('base.db')
            cursor = conexion.cursor()

            # Crear la tabla 'bitcoin' si no existe
            cursor.execute('''CREATE TABLE IF NOT EXISTS bitcoin (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                fecha DATE,
                                precio_usd DECIMAL(10, 2),
                                precio_gbp DECIMAL(10, 2),
                                precio_eur DECIMAL(10, 2),
                                precio_pen DECIMAL(10, 2)
                            )''')

            # Insertar los datos en la tabla 'bitcoin'
            fecha = '2024-02-14'  # Fecha de los datos
            precio_usd = precios_bitcoin['USD']['rate_float']
            precio_gbp = precios_bitcoin['GBP']['rate_float']
            precio_eur = precios_bitcoin['EUR']['rate_float']
            precio_pen = precio_usd * tipo_cambio_pen

            cursor.execute('''INSERT INTO bitcoin (fecha, precio_usd, precio_gbp, precio_eur, precio_pen) 
                              VALUES (?, ?, ?, ?, ?)''', (fecha, precio_usd, precio_gbp, precio_eur, precio_pen))
            
            conexion.commit()
            print("Datos almacenados correctamente en la tabla 'bitcoin'.")
        
        except sqlite3.Error as error:
            print("Error al trabajar con la base de datos:", error)
        finally:
            if conexion:
                conexion.close()
    else:
        print("No se pudieron obtener los datos necesarios.")

# Función para calcular el precio de comprar 10 bitcoins en moneda PEN y EUR
def calcular_precio_compra_10_bitcoins():
    try:
        conexion = sqlite3.connect('base.db')
        cursor = conexion.cursor()

        # Consultar el precio de comprar 10 bitcoins en PEN y EUR
        cursor.execute("SELECT precio_pen, precio_eur FROM bitcoin ORDER BY fecha DESC LIMIT 1")
        resultado = cursor.fetchone()
        if resultado:
            precio_compra_pen = resultado[0] * 10
            precio_compra_eur = resultado[1] * 10
            print(f"Precio de comprar 10 bitcoins en PEN: {precio_compra_pen}")
            print(f"Precio de comprar 10 bitcoins en EUR: {precio_compra_eur}")
        else:
            print("No se encontraron datos en la tabla 'bitcoin'.")
    
    except sqlite3.Error as error:
        print("Error al trabajar con la base de datos:", error)
    finally:
        if conexion:
            conexion.close()

def main():
    # Crear la tabla 'bitcoin' y almacenar los datos
    crear_tabla_y_almacenar_datos()

    # Calcular el precio de comprar 10 bitcoins en PEN y EUR
    calcular_precio_compra_10_bitcoins()